## API client

`api.API` is a thin wrapper around the [HERE search & geocoding API](https://developer.here.com/documentation/geocoding-search-api/api-reference-swagger.html). 
API objects host the `api_key` variable (prompted if not provided as argument or `API_KEY` environment variable).

In [1]:
from here_search.api import API

api=API()

The API class uses async methods to take advantage of Jupyter event loop and [async REPL](https://blog.jupyter.org/ipython-7-0-async-repl-a35ce050f7f7). For example `/discover` requests are sent with `api.discover()` like:

In [2]:
resp = await api.discover(q="berlin", latitude=52, longitude=13)
resp.req.full

'https://discover.search.hereapi.com/v1/discover?q=berlin&at=52%2C13&apiKey=tIEID-Z-ogljOD83iQO6JiHbGtn9lyFmt0srqjIRtao'

In [3]:
resp.data

{'items': [{'title': 'Berlin, Deutschland',
   'id': 'here:cm:namedplace:20187403',
   'language': 'de',
   'resultType': 'locality',
   'localityType': 'city',
   'address': {'label': 'Berlin, Deutschland',
    'countryCode': 'DEU',
    'countryName': 'Deutschland',
    'stateCode': 'BE',
    'state': 'Berlin',
    'countyCode': 'B',
    'county': 'Berlin',
    'city': 'Berlin',
    'postalCode': '10117'},
   'position': {'lat': 52.51604, 'lng': 13.37691},
   'distance': 62854,
   'mapView': {'west': 13.08835,
    'south': 52.33812,
    'east': 13.761,
    'north': 52.6755}}]}

In [6]:
resp = await api.autosuggest(q="restaura", latitude=52, longitude=13, limit=2, termsLimit=3)
resp.req.full

'https://autosuggest.search.hereapi.com/v1/autosuggest?q=restaura&at=52%2C13&limit=2&termsLimit=3&apiKey=tIEID-Z-ogljOD83iQO6JiHbGtn9lyFmt0srqjIRtao'

In [7]:
resp.data

{'items': [{'title': 'Restaurant',
   'id': 'here:cm:ontology:restaurant',
   'resultType': 'categoryQuery',
   'href': 'https://autosuggest.search.hereapi.com/v1/discover?q=Restaurant&_ontology=restaurant&at=52.0%2C13.0',
   'highlights': {'title': [{'start': 0, 'end': 8}]}},
  {'title': 'Restaurant Alte Post',
   'id': 'here:pds:place:276u31pz-5d3dd22e28f04340c806c9d6e3491260',
   'resultType': 'place',
   'address': {'label': 'Restaurant Alte Post, Treuenbrietzener Straße 45, 14913 Niedergörsdorf, Deutschland'},
   'position': {'lat': 52.01699, 'lng': 12.99429},
   'access': [{'lat': 52.01682, 'lng': 12.99409}],
   'distance': 1929,
   'categories': [{'id': '100-1000-0006',
     'name': 'Delikatessen',
     'primary': True},
    {'id': '100-1000-0000', 'name': 'Restaurant'},
    {'id': '100-1100-0010', 'name': 'Kaffeehaus'}],
   'references': [{'supplier': {'id': 'tripadvisor'}, 'id': '23950733'}],
   'foodTypes': [{'id': '300-000', 'name': 'Europäisch', 'primary': True},
    {'id':

In [8]:
resp.terms

['Restaurant', 'Restauration']

In [9]:
resp.geojson()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [12.99429, 52.01699]},
   'properties': {'title': 'Restaurant Alte Post',
    'categories': 'Delikatessen'}}]}

## Suggestions for a growing query

Autosuggest is made to help end-user to quickly formulate a query.


It is recommended to use Autosuggest response items `resulType` field to decide what to do in case of user selection. The selection of a `chainQuery` or `categoryQuery` result should lead to a GET of the `href` field value. Other selections should lead to a call to `/lookup` using the `id` field value. 
Note that the `resulType` field can be used to render results differently. For example, a <img src="https://upload.wikimedia.org/wikipedia/commons/2/2b/Font_Awesome_5_solid_search.svg" style="width:12px"/> can be used to signal a `chainQuery` or `categoryQuery`.

The following snippet sends an Autosuggets request for each additional character of the query "restaurant hamburg" a hypothetic user intends to type to get restaurants near the German city of Hamburg. Only each response first result is displayed: 

In [27]:
q = "restaurant hamburg"

template = "{:<2} {:<25} {:<18} {:<65} {:<14}"
print(template.format("#", "query", "resultType", "id", "title"))
for i in range(len(q)):
    resp = await api.autosuggest(q=q[:i+1], latitude=52, longitude=13,  lang="en", limit=1)
    item = resp.data["items"][0]
    print(template.format(i+1, f"'{q[:i+1]}'", item['resultType'], item['id'][:60], item['title']))

#  query                     resultType         id                                                                title         
1  'r'                       administrativeArea here:cm:namedplace:20181414                                       Regensburg (Landkreis) (R), Bavaria, Germany
2  're'                      locality           here:cm:namedplace:20180814                                       Regensburg, Bavaria, Germany
3  'res'                     place              here:pds:place:276u33d2-49b7fa2d1beb4df6bd489533a371dbc0          IKEA Restaurant
4  'rest'                    categoryQuery      here:cm:taxonomy:restaurant                                       Restaurant    
5  'resta'                   categoryQuery      here:cm:taxonomy:restaurant                                       Restaurant    
6  'restau'                  categoryQuery      here:cm:taxonomy:restaurant                                       Restaurant    
7  'restaur'                 categoryQuery      here

If the end-user was actually typing the query "restaurant", the suggestion of `id` value `here:cm:taxonomy:restaurant` returned for the 4-letter query "rest" could be selected to trigger the related `href` query. But the end-user needs to continue typing untill "restaurants ham" to get the suggestion titled "Restaurants near Hamburg, Germany".

## Terms suggestions

To further help end-user to formulate such a long query, Autosuggest also returns predictive text for the query last token while it is being typed. Those suggested terms are in the response `queryTerms` when the request contains a positive `termsLimit` parameter. In the snippet below, 3 last token suggestiosn are returned per response:

In [28]:
template = "{:<2} {:<25} {:<38} {:<14}"
print(template.format("#", "query", "terms", "title"))
for i in range(len(q)):
    resp = await api.autosuggest(q=q[:i+1], latitude=52, longitude=13, lang="en", limit=1, termsLimit=3)
    item = resp.data["items"][0]
    terms = ",".join(t["term"] for t in resp.data["queryTerms"])
    print(template.format(i+1, f"'{q[:i+1]}'", terms, item['title']))

#  query                     terms                                  title         
1  'r'                       Raum,Roubroeks                         Regensburg (Landkreis) (R), Bavaria, Germany
2  're'                      Regensburg,Recife,Region               Regensburg, Bavaria, Germany
3  'res'                     Restaurant,Restauration,Resort         IKEA Restaurant
4  'rest'                    Restaurant,Restroom,Restaurant         Restaurant    
5  'resta'                   Restaurant,Restauration,Restaurant     Restaurant    
6  'restau'                  Restaurant,Restauration,Restaurante    Restaurant    
7  'restaur'                 Restaurant,Restaurant,Restauration     Restaurant    
8  'restaura'                Restaurant,Restaurant,Restauration     Restaurant    
9  'restauran'               Restaurant,Restaurant                  Restaurant    
10 'restaurant'              Restaurante                            Restaurant    
11 'restaurant '             Restaurante  

When presented to the end-user to replace the last token in the query, `queryTerms` can significantly speed-up the query submission. In the previous interaction, the use of the token "Restaurant" returned for the "res" query, would have led to the series of 8 queries instead of 14: "r", "re", "res", "restaurant", "restaurant ", "restaurant h", "restaurant ha", "restaurant hamburg".

In [32]:
template = "{:<2} {:<25} {:<38} {:<14}"
print(template.format("#", "query", "terms", "title"))
qs = ["r", "re", "res", "restaurant", "restaurant ", "restaurant h", "restaurant ha", "restaurant hamburg"]
for i, q in enumerate(qs):
    resp = await api.autosuggest(q=q, latitude=52, longitude=13, lang="en", limit=1, termsLimit=3)
    item = resp.data["items"][0]
    terms = ",".join(t["term"] for t in resp.data["queryTerms"])
    print(template.format(i+1, f"'{q}'", terms, item['title']))

#  query                     terms                                  title         
1  'r'                       Raum,Roubroeks                         Regensburg (Landkreis) (R), Bavaria, Germany
2  're'                      Regensburg,Recife,Region               Regensburg, Bavaria, Germany
3  'res'                     Restaurant,Restauration,Resort         IKEA Restaurant
4  'restaurant'              Restaurante                            Restaurant    
5  'restaurant '             Restaurante                            Restaurant    
6  'restaurant h'            Hopfingerbräu,Havellandhalle,Hanover   Hopfingerbräu im Palais
7  'restaurant ha'           Hammermühle,Hamburg,Hawally            Gasthof Hammermuhle (Restaurant Hammermühle)
8  'restaurant hamburg'      Hamburger                              Restaurant near Hamburg, Germany
